In [67]:
# import packages
import pandas as pd
import json
#from flat_table import normalize
import ast
import re
import math
import numpy as np

In [68]:
# load the package
final_df = pd.read_csv('/project/all_data.csv')
final_df.head()

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,abstract.text,lens_id,jurisdiction,kind,date_published,publication_type,biblio.classifications_cpc.classifications,claims.claims,biblio.invention_title.text,biblio.invention_title.lang,biblio.invention_title
0,0,0,NaN,000-232-863-605-99X,US,S,2016-03-29,DESIGN_RIGHT,NaN,NaN,Toy,en,NaN
1,1,1,NaN,000-480-956-918-728,US,S,2016-02-09,DESIGN_RIGHT,NaN,NaN,Signal indicator lamp,en,NaN
2,2,2,NaN,001-158-853-914-602,US,S,2016-03-01,DESIGN_RIGHT,NaN,NaN,Package for a medicinal product,en,NaN
3,3,3,NaN,001-188-401-116-076,US,S,2016-03-15,DESIGN_RIGHT,NaN,NaN,Name plate for a tabletop,en,NaN
4,4,4,"Systems, methods, and apparatuses are provided...",001-215-454-810-863,US,B2,2016-03-15,GRANTED_PATENT,"A61N5/1049,A61N5/1049,A61B5/0036,A61B5/0036,A6...",1. A system comprising: an imaging element con...,Imaging observation timing for assisting radia...,en,NaN


In [69]:
# final_df = final_df[final_df['biblio.invention_title.text'].notna()]
final_df.head()

,Unnamed: 0,Unnamed: 0.1,abstract.text,lens_id,jurisdiction,kind,date_published,publication_type,biblio.classifications_cpc.classifications,claims.claims,biblio.invention_title.text,biblio.invention_title.lang,biblio.invention_title
0,0,0,NaN,000-232-863-605-99X,US,S,2016-03-29,DESIGN_RIGHT,NaN,NaN,Toy,en,NaN
1,1,1,NaN,000-480-956-918-728,US,S,2016-02-09,DESIGN_RIGHT,NaN,NaN,Signal indicator lamp,en,NaN
2,2,2,NaN,001-158-853-914-602,US,S,2016-03-01,DESIGN_RIGHT,NaN,NaN,Package for a medicinal product,en,NaN
3,3,3,NaN,001-188-401-116-076,US,S,2016-03-15,DESIGN_RIGHT,NaN,NaN,Name plate for a tabletop,en,NaN
4,4,4,"Systems, methods, and apparatuses are provided...",001-215-454-810-863,US,B2,2016-03-15,GRANTED_PATENT,"A61N5/1049,A61N5/1049,A61B5/0036,A61B5/0036,A6...",1. A system comprising: an imaging element con...,Imaging observation timing for assisting radia...,en,NaN


In [70]:
final_df = final_df.drop(columns=['biblio.invention_title','biblio.invention_title.lang'])
# drop column
final_df = final_df.drop(columns=['Unnamed: 0'])
final_df = final_df.drop(columns=['Unnamed: 0.1'])

final_df.isna().sum()

abstract.text                                  28908
lens_id                                            0
jurisdiction                                       0
kind                                               0
date_published                                     0
publication_type                                   0
biblio.classifications_cpc.classifications     29174
claims.claims                                  28876
biblio.invention_title.text                   178577
dtype: int64

In [71]:
# check cpc null values for "DESIGN RIGHT"
dr = final_df[final_df['publication_type'] == "DESIGN_RIGHT"]
dr.shape

(28874, 9)

In [72]:
# drop null values
final_df['cpc'] = final_df['biblio.classifications_cpc.classifications'].dropna(axis=0,how='any')
final_df = final_df[final_df['cpc'].notna()]
final_df.isna().sum()

abstract.text                                    349
lens_id                                            0
jurisdiction                                       0
kind                                               0
date_published                                     0
publication_type                                   0
biblio.classifications_cpc.classifications         0
claims.claims                                    350
biblio.invention_title.text                   170971
cpc                                                0
dtype: int64

In [73]:
# # replace 
# final_df['biblio.invention_title'].replace(np.nan,'unknown', regex=True)
# final_df['abstract.text'].replace(np.nan,'unknown', regex=True)

In [74]:
# binary classification for cpc
final_df["yo2"]= final_df["biblio.classifications_cpc.classifications"].apply(lambda series: 1 if series.find('Y02')!=-1 else 0)

In [75]:
final_df.head()

,abstract.text,lens_id,jurisdiction,kind,date_published,publication_type,biblio.classifications_cpc.classifications,claims.claims,biblio.invention_title.text,cpc,yo2
4,"Systems, methods, and apparatuses are provided...",001-215-454-810-863,US,B2,2016-03-15,GRANTED_PATENT,"A61N5/1049,A61N5/1049,A61B5/0036,A61B5/0036,A6...",1. A system comprising: an imaging element con...,Imaging observation timing for assisting radia...,"A61N5/1049,A61N5/1049,A61B5/0036,A61B5/0036,A6...",0
8,Memoizable functions may be identified by anal...,002-479-253-838-718,US,B2,2016-02-16,GRANTED_PATENT,"G06F16/2308,G06F8/4441,G06F11/3616,G06F16/2455...",1. A computer-implemented method of performing...,Purity analysis using white list/black list an...,"G06F16/2308,G06F8/4441,G06F11/3616,G06F16/2455...",0
9,This system provides wired and/or wireless acc...,002-813-480-436-824,US,B2,2016-03-29,GRANTED_PATENT,"G06F21/10,G06F21/10,G06F16/40,G06F21/6218,G06F...",1. A multimedia management system for managing...,Multimedia network system with content importa...,"G06F21/10,G06F21/10,G06F16/40,G06F21/6218,G06F...",0
10,"A new, distinct Geum plant as shown and descri...",004-133-132-043-147,US,P2,2016-03-01,PLANT_PATENT,"A01H6/74,A01H5/02,","1. A new, distinct Geum plant as illustrated a...",Geum plant named ‘Sea Breeze’,"A01H6/74,A01H5/02,",0
16,An automated banking machine controlled respon...,006-135-481-468-129,US,B2,2016-03-01,GRANTED_PATENT,"G07F19/20,G07F19/20,G06Q40/02,G06Q40/02,G07D7/...","1. A tangible, non-transitory computer readabl...",Automated banking machine that operates respon...,"G07F19/20,G07F19/20,G06Q40/02,G06Q40/02,G07D7/...",0


In [76]:
final_df.isna().sum()

abstract.text                                    349
lens_id                                            0
jurisdiction                                       0
kind                                               0
date_published                                     0
publication_type                                   0
biblio.classifications_cpc.classifications         0
claims.claims                                    350
biblio.invention_title.text                   170971
cpc                                                0
yo2                                                0
dtype: int64

In [77]:
# fill null values with "unknown"
final_df['biblio.invention_title.text'] = final_df['biblio.invention_title.text'].fillna('unknown')
final_df['abstract.text'] = final_df['abstract.text'].fillna('unknown')
final_df['claims.claims'] = final_df['claims.claims'].fillna('unknown')

In [78]:
final_df.isna().sum()

abstract.text                                 0
lens_id                                       0
jurisdiction                                  0
kind                                          0
date_published                                0
publication_type                              0
biblio.classifications_cpc.classifications    0
claims.claims                                 0
biblio.invention_title.text                   0
cpc                                           0
yo2                                           0
dtype: int64

In [79]:
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

In [80]:
# lower case variables
def clean_lowercase(review_text):
    return str(review_text).lower()
final_df['abstract.text_lower'] = final_df['abstract.text'].apply(clean_lowercase)
final_df['abstract.text_lower']

4         systems, methods, and apparatuses are provided...
8         memoizable functions may be identified by anal...
9         this system provides wired and/or wireless acc...
10        a new, distinct geum plant as shown and descri...
16        an automated banking machine controlled respon...
                                ...                        
716449    a tape cassette that includes a housing having...
716450    laser 3d imaging techniques include splitting ...
716451    a cooling apparatus includes a coolant circula...
716452    in one embodiment, an apparatus includes: a ho...
716453    embodiments of the present disclosure are dire...
Name: abstract.text_lower, Length: 687280, dtype: object

In [81]:
# removing punctuation
def remove_before_token(sentence, keep_apostrophe = False):
    sentence = sentence.strip()
    if keep_apostrophe:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r' ', sentence)
    else :
        PATTERN = r'[^a-zA-Z0-9]'
        filtered_sentence = re.sub(PATTERN, r' ', sentence)
    return(filtered_sentence)

final_df['abstract.text_punctbef'] = final_df['abstract.text_lower'].apply(remove_before_token)
final_df['abstract.text_punctbef']

4         systems  methods  and apparatuses are provided...
8         memoizable functions may be identified by anal...
9         this system provides wired and or wireless acc...
10        a new  distinct geum plant as shown and descri...
16        an automated banking machine controlled respon...
                                ...                        
716449    a tape cassette that includes a housing having...
716450    laser 3d imaging techniques include splitting ...
716451    a cooling apparatus includes a coolant circula...
716452    in one embodiment  an apparatus includes  a ho...
716453    embodiments of the present disclosure are dire...
Name: abstract.text_punctbef, Length: 687280, dtype: object

In [82]:
# remove numbers
def clean_text(text):
    text_nonum = re.sub(r'\d+', '', text)
    return text_nonum

final_df['abstract.text_no'] = final_df['abstract.text_punctbef'].apply(clean_text)
final_df['abstract.text_no']

4         systems  methods  and apparatuses are provided...
8         memoizable functions may be identified by anal...
9         this system provides wired and or wireless acc...
10        a new  distinct geum plant as shown and descri...
16        an automated banking machine controlled respon...
                                ...                        
716449    a tape cassette that includes a housing having...
716450    laser d imaging techniques include splitting a...
716451    a cooling apparatus includes a coolant circula...
716452    in one embodiment  an apparatus includes  a ho...
716453    embodiments of the present disclosure are dire...
Name: abstract.text_no, Length: 687280, dtype: object

In [83]:
from nltk.corpus import stopwords
#stopwords.words('english')
stop_words = stopwords.words('english')

def clean_stopwords(document):
    return' '.join([word for word in document.split() if word not in stop_words])

final_df['abstract.text_sw'] = final_df['abstract.text_no'].apply(clean_stopwords)
final_df['abstract.text_sw']

4         systems methods apparatuses provided targeting...
8         memoizable functions may identified analyzing ...
9         system provides wired wireless access througho...
10        new distinct geum plant shown described charac...
16        automated banking machine controlled responsiv...
                                ...                        
716449    tape cassette includes housing top surface bot...
716450    laser imaging techniques include splitting las...
716451    cooling apparatus includes coolant circulation...
716452    one embodiment apparatus includes housing incl...
716453    embodiments present disclosure directed system...
Name: abstract.text_sw, Length: 687280, dtype: object

In [84]:
final_df.head()

,abstract.text,lens_id,jurisdiction,kind,date_published,publication_type,biblio.classifications_cpc.classifications,claims.claims,biblio.invention_title.text,cpc,yo2,abstract.text_lower,abstract.text_punctbef,abstract.text_no,abstract.text_sw
4,"Systems, methods, and apparatuses are provided...",001-215-454-810-863,US,B2,2016-03-15,GRANTED_PATENT,"A61N5/1049,A61N5/1049,A61B5/0036,A61B5/0036,A6...",1. A system comprising: an imaging element con...,Imaging observation timing for assisting radia...,"A61N5/1049,A61N5/1049,A61B5/0036,A61B5/0036,A6...",0,"systems, methods, and apparatuses are provided...",systems methods and apparatuses are provided...,systems methods and apparatuses are provided...,systems methods apparatuses provided targeting...
8,Memoizable functions may be identified by anal...,002-479-253-838-718,US,B2,2016-02-16,GRANTED_PATENT,"G06F16/2308,G06F8/4441,G06F11/3616,G06F16/2455...",1. A computer-implemented method of performing...,Purity analysis using white list/black list an...,"G06F16/2308,G06F8/4441,G06F11/3616,G06F16/2455...",0,memoizable functions may be identified by anal...,memoizable functions may be identified by anal...,memoizable functions may be identified by anal...,memoizable functions may identified analyzing ...
9,This system provides wired and/or wireless acc...,002-813-480-436-824,US,B2,2016-03-29,GRANTED_PATENT,"G06F21/10,G06F21/10,G06F16/40,G06F21/6218,G06F...",1. A multimedia management system for managing...,Multimedia network system with content importa...,"G06F21/10,G06F21/10,G06F16/40,G06F21/6218,G06F...",0,this system provides wired and/or wireless acc...,this system provides wired and or wireless acc...,this system provides wired and or wireless acc...,system provides wired wireless access througho...
10,"A new, distinct Geum plant as shown and descri...",004-133-132-043-147,US,P2,2016-03-01,PLANT_PATENT,"A01H6/74,A01H5/02,","1. A new, distinct Geum plant as illustrated a...",Geum plant named ‘Sea Breeze’,"A01H6/74,A01H5/02,",0,"a new, distinct geum plant as shown and descri...",a new distinct geum plant as shown and descri...,a new distinct geum plant as shown and descri...,new distinct geum plant shown described charac...
16,An automated banking machine controlled respon...,006-135-481-468-129,US,B2,2016-03-01,GRANTED_PATENT,"G07F19/20,G07F19/20,G06Q40/02,G06Q40/02,G07D7/...","1. A tangible, non-transitory computer readabl...",Automated banking machine that operates respon...,"G07F19/20,G07F19/20,G06Q40/02,G06Q40/02,G07D7/...",0,an automated banking machine controlled respon...,an automated banking machine controlled respon...,an automated banking machine controlled respon...,automated banking machine controlled responsiv...


In [85]:
# TEXT PROCESSING FOR TITLE

In [86]:
# lower case variables for title 
def clean_lowercase(review_text):
    return str(review_text).lower()
final_df['biblio.invention_title.text_lower'] = final_df['biblio.invention_title.text'].apply(clean_lowercase)
final_df['biblio.invention_title.text_lower']

4         imaging observation timing for assisting radia...
8         purity analysis using white list/black list an...
9         multimedia network system with content importa...
10                            geum plant named ‘sea breeze’
16        automated banking machine that operates respon...
                                ...                        
716449                                              unknown
716450                                              unknown
716451                                              unknown
716452                                              unknown
716453                                              unknown
Name: biblio.invention_title.text_lower, Length: 687280, dtype: object

In [87]:
# removing punctuation for title
def remove_before_token(sentence, keep_apostrophe = False):
    sentence = sentence.strip()
    if keep_apostrophe:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r' ', sentence)
    else :
        PATTERN = r'[^a-zA-Z0-9]'
        filtered_sentence = re.sub(PATTERN, r' ', sentence)
    return(filtered_sentence)

final_df['biblio.invention_title.text_punctbef'] = final_df['biblio.invention_title.text_lower'].apply(remove_before_token)
final_df['biblio.invention_title.text_punctbef']

4         imaging observation timing for assisting radia...
8         purity analysis using white list black list an...
9         multimedia network system with content importa...
10                            geum plant named  sea breeze 
16        automated banking machine that operates respon...
                                ...                        
716449                                              unknown
716450                                              unknown
716451                                              unknown
716452                                              unknown
716453                                              unknown
Name: biblio.invention_title.text_punctbef, Length: 687280, dtype: object

In [88]:
# remove numbers
def clean_text(text):
    text_nonum = re.sub(r'\d+', '', text)
    return text_nonum

final_df['biblio.invention_title.text_no'] = final_df['biblio.invention_title.text_punctbef'].apply(clean_text)
final_df['biblio.invention_title.text_no']

4         imaging observation timing for assisting radia...
8         purity analysis using white list black list an...
9         multimedia network system with content importa...
10                            geum plant named  sea breeze 
16        automated banking machine that operates respon...
                                ...                        
716449                                              unknown
716450                                              unknown
716451                                              unknown
716452                                              unknown
716453                                              unknown
Name: biblio.invention_title.text_no, Length: 687280, dtype: object

In [89]:
# remove numbers
def clean_text(text):
    text_nonum = re.sub(r'\d+', '', text)
    return text_nonum

final_df['biblio.invention_title.text_sw'] = final_df['biblio.invention_title.text_no'].apply(clean_text)
final_df['biblio.invention_title.text_sw']

4         imaging observation timing for assisting radia...
8         purity analysis using white list black list an...
9         multimedia network system with content importa...
10                            geum plant named  sea breeze 
16        automated banking machine that operates respon...
                                ...                        
716449                                              unknown
716450                                              unknown
716451                                              unknown
716452                                              unknown
716453                                              unknown
Name: biblio.invention_title.text_sw, Length: 687280, dtype: object

In [90]:
# TEXT PROCESSING FOR CLAIMS

In [91]:
# lower case variables for claims 
def clean_lowercase(review_text):
    return str(review_text).lower()
final_df['claims.claims_lower'] = final_df['claims.claims'].apply(clean_lowercase)
final_df['claims.claims_lower']

4         1. a system comprising: an imaging element con...
8         1. a computer-implemented method of performing...
9         1. a multimedia management system for managing...
10        1. a new, distinct geum plant as illustrated a...
16        1. a tangible, non-transitory computer readabl...
                                ...                        
716449    1. a tape cassette, comprising: a housing havi...
716450    1 . a method comprising: a) splitting a laser ...
716451    1 . a cooling apparatus for an internal combus...
716452    1 . an apparatus comprising: a housing includi...
716453    1 . a method comprising: accessing at least on...
Name: claims.claims_lower, Length: 687280, dtype: object

In [92]:
# removing punctuation for claims
def remove_before_token(sentence, keep_apostrophe = False):
    sentence = sentence.strip()
    if keep_apostrophe:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r' ', sentence)
    else :
        PATTERN = r'[^a-zA-Z0-9]'
        filtered_sentence = re.sub(PATTERN, r' ', sentence)
    return(filtered_sentence)

final_df['claims.claims_punctbef'] = final_df['claims.claims_lower'].apply(remove_before_token)
final_df['claims.claims_punctbef']

4         1  a system comprising  an imaging element con...
8         1  a computer implemented method of performing...
9         1  a multimedia management system for managing...
10        1  a new  distinct geum plant as illustrated a...
16        1  a tangible  non transitory computer readabl...
                                ...                        
716449    1  a tape cassette  comprising  a housing havi...
716450    1   a method comprising  a  splitting a laser ...
716451    1   a cooling apparatus for an internal combus...
716452    1   an apparatus comprising  a housing includi...
716453    1   a method comprising  accessing at least on...
Name: claims.claims_punctbef, Length: 687280, dtype: object

In [93]:
# remove numbers
def clean_text(text):
    text_nonum = re.sub(r'\d+', '', text)
    return text_nonum

final_df['claims.claims_no'] = final_df['claims.claims_punctbef'].apply(clean_text)
final_df['claims.claims_no']

4           a system comprising  an imaging element conf...
8           a computer implemented method of performing ...
9           a multimedia management system for managing ...
10          a new  distinct geum plant as illustrated an...
16          a tangible  non transitory computer readable...
                                ...                        
716449      a tape cassette  comprising  a housing havin...
716450       a method comprising  a  splitting a laser t...
716451       a cooling apparatus for an internal combust...
716452       an apparatus comprising  a housing includin...
716453       a method comprising  accessing at least one...
Name: claims.claims_no, Length: 687280, dtype: object

In [94]:
# remove numbers
def clean_text(text):
    text_nonum = re.sub(r'\d+', '', text)
    return text_nonum

final_df['claims.claims_sw'] = final_df['claims.claims_no'].apply(clean_text)
final_df['claims.claims_sw']

4           a system comprising  an imaging element conf...
8           a computer implemented method of performing ...
9           a multimedia management system for managing ...
10          a new  distinct geum plant as illustrated an...
16          a tangible  non transitory computer readable...
                                ...                        
716449      a tape cassette  comprising  a housing havin...
716450       a method comprising  a  splitting a laser t...
716451       a cooling apparatus for an internal combust...
716452       an apparatus comprising  a housing includin...
716453       a method comprising  accessing at least one...
Name: claims.claims_sw, Length: 687280, dtype: object

In [95]:
# drop columns
#final_df = final_df.drop(columns=['unnamed'])
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687280 entries, 4 to 716453
Data columns (total 23 columns):
 #   Column                                      Non-Null Count   Dtype 
---  ------                                      --------------   ----- 
 0   abstract.text                               687280 non-null  object
 1   lens_id                                     687280 non-null  object
 2   jurisdiction                                687280 non-null  object
 3   kind                                        687280 non-null  object
 4   date_published                              687280 non-null  object
 5   publication_type                            687280 non-null  object
 6   biblio.classifications_cpc.classifications  687280 non-null  object
 7   claims.claims                               687280 non-null  object
 8   biblio.invention_title.text                 687280 non-null  object
 9   cpc                                         687280 non-null  object
 10  yo2     

In [96]:
# final_df
final_df = final_df.drop(columns=['abstract.text','claims.claims','jurisdiction','kind','biblio.invention_title.text','biblio.classifications_cpc.classifications',
                                 'abstract.text_lower','abstract.text_punctbef','abstract.text_no',
                                 'biblio.invention_title.text_lower','biblio.invention_title.text_punctbef','biblio.invention_title.text_no',
                                 'claims.claims_lower','claims.claims_punctbef','claims.claims_no'])


In [97]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687280 entries, 4 to 716453
Data columns (total 8 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   lens_id                         687280 non-null  object
 1   date_published                  687280 non-null  object
 2   publication_type                687280 non-null  object
 3   cpc                             687280 non-null  object
 4   yo2                             687280 non-null  int64 
 5   abstract.text_sw                687280 non-null  object
 6   biblio.invention_title.text_sw  687280 non-null  object
 7   claims.claims_sw                687280 non-null  object
dtypes: int64(1), object(7)
memory usage: 47.2+ MB


In [98]:
# tokenize
import nltk
from nltk.tokenize import word_tokenize

def clean_tokenization(review_text):
    return word_tokenize(review_text)

final_df['biblio.invention_title.text_token'] = final_df['biblio.invention_title.text_sw'].apply(clean_tokenization)
final_df['biblio.invention_title.text_token']

4         [imaging, observation, timing, for, assisting,...
8         [purity, analysis, using, white, list, black, ...
9         [multimedia, network, system, with, content, i...
10                        [geum, plant, named, sea, breeze]
16        [automated, banking, machine, that, operates, ...
                                ...                        
716449                                            [unknown]
716450                                            [unknown]
716451                                            [unknown]
716452                                            [unknown]
716453                                            [unknown]
Name: biblio.invention_title.text_token, Length: 687280, dtype: object

In [99]:
# final_df.to_csv('final_preprocessing.csv')

In [100]:
final_df.to_csv('final_preprocessing_wk.csv')